In [ ]:
from IPython.display import display 
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
import plotly.graph_objects as go
import random

import scikitplot as skplt
from sklearn.metrics import accuracy_score, classification_report, f1_score,roc_auc_score,accuracy_score,confusion_matrix
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
import optuna
from optuna.samplers import TPESampler
import pickle
import warnings
from optuna import Trial
import collections
warnings.filterwarnings('ignore')

In [ ]:
def preprocessing_training(adataPath, hvgs):
    # read data
    adata = sc.read_10x_mtx(adataPath,var_names= 'gene_symbols',cache = False)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    # Filter HVGs --> Select top N highly variable genes that will serve as features to the machine learning models  
    sc.pp.highly_variable_genes(adata, n_top_genes = hvgs)
    highly_variable_genes = adata.var["highly_variable"]
    highly_variable_genes_names = adata.var_names[adata.var['highly_variable']].to_list()
    adata = adata[:, highly_variable_genes]
    # Scale
    sc.pp.scale(adata, max_value=10)
    X_df = pd.DataFrame(adata.X, index=adata.obs_names, columns=adata.var_names)
    print(type(X_df), X_df.shape)
    return X_df, highly_variable_genes_names

root_df, highly_variable_genes  = preprocessing_training(adataPath = "/public/home/lidongwei/work/scRNA_scATAC/part_sc_reference_atlas/data/merge_data/matrix_info",
                                    hvgs = 2000)

In [ ]:
cell_type = pd.read_csv("/public/home/lidongwei/work/scRNA_scATAC/part_sc_reference_atlas/data/merge_data/wxy_wjw_lq_merge_meta_231121.csv",index_col=0)

root_df = root_df.sort_index()
cell_type = cell_type.sort_index()

cell_type_label = cell_type['cell_type'].values
print(type(cell_type_label), cell_type_label.shape)
print('List of unique labels we wish to transfer: {}'.format(np.unique(cell_type_label)))
print(collections.Counter(cell_type['cell_type']))

unique_labels, label_counts = np.unique(cell_type_label, return_counts=True)

plt.bar(unique_labels, label_counts/len(cell_type_label))
plt.title("Proportion of Labels")
plt.xlabel("Labels")
plt.ylabel("Proportion")
plt.xticks(rotation=90)
plt.show()    

In [ ]:

def downsample_data(count, raw_label, threshold):
    label = pd.DataFrame(raw_label)
    data = pd.concat([count, label], axis=1)

    sample_counts = label['cell_type'].value_counts().to_dict()

    downsampled_data = pd.DataFrame()  

    for label, count in sample_counts.items():
        if count > threshold:
            samples = data[data['cell_type'] == label].sample(n=threshold, random_state=1)
            downsampled_data = pd.concat([downsampled_data, samples])  
        else:
            samples = data[data['cell_type'] == label]
            downsampled_data = pd.concat([downsampled_data, samples])

    features = downsampled_data.drop('cell_type', axis=1)  
    labels = downsampled_data['cell_type']  

    return features, labels

root_df_down,cell_type_label_down = downsample_data(root_df,cell_type['cell_type'],5000)

print(collections.Counter(cell_type_label_down))
print(root_df_down.shape)

In [ ]:
def data_pro(x,y):
    x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2,random_state=33,stratify=y)
    return x_train, x_test, y_train, y_test

# Split the training dataset into train and test sets 
X_train, X_test, y_train, y_test = data_pro(root_df_down,cell_type_label_down)

In [ ]:
class Objective(object):
    def __init__(self, x_calib, y_calib, cv_folds,use_thread_count,use_task,use_device, scoring_type):
        self.x_calib = x_calib
        self.y_calib = y_calib
        self.use_thread_count = use_thread_count
        self.use_task = use_task
        self.use_device = use_device
        self.scoring_type = scoring_type
        self.cv_folds = cv_folds
 
    def __call__(self, trial):
         
        params = {'iterations':trial.suggest_int("iterations", 100, 1000),
              'loss_function':'MultiClass',
              'boosting_type':'Plain',
              'custom_metric':'Accuracy',
              'eval_metric':'TotalF1',
              'leaf_estimation_method':'Newton',
              'bootstrap_type': 'Bernoulli',
              'learning_rate' : trial.suggest_uniform('learning_rate',1e-3, 1),
              'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
              'random_strength': trial.suggest_uniform('random_strength', 1e-3, 10.0),
              'depth': trial.suggest_int('depth',4,13),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,50),
              'auto_class_weights':trial.suggest_categorical('auto_class_weights', [None,'Balanced'])
               }
    
        if self.use_task == 'GPU':
            classifier_obj = CatBoostClassifier(**params,task_type='GPU',thread_count=self.use_thread_count,
                                                devices=self.use_device,verbose=False,allow_writing_files=False)
        elif self.use_task == 'CPU':
            classifier_obj = CatBoostClassifier(**params,task_type='CPU',thread_count=self.use_thread_count,
                                                verbose=False,allow_writing_files=False) 
        score = cross_val_score(classifier_obj, self.x_calib, self.y_calib, 
                                cv=StratifiedKFold(n_splits=self.cv_folds, shuffle=True, random_state=1), 
                                scoring=self.scoring_type, n_jobs=self.use_thread_count)
        mean_cv_score = score.mean()
        
        return mean_cv_score

In [ ]:
scoring_type =  'accuracy'
optimizer_direction = 'maximize'
cv_folds = 10
number_of_random_points = 15 # defualt 10
x_calib = X_train
y_calib = y_train
use_thread_count = 1
use_task = 'GPU'
use_device = '0:1'

optuna_objective = Objective(x_calib, y_calib, cv_folds,use_thread_count,use_task,use_device, scoring_type)
 
optuna.logging.set_verbosity(optuna.logging.WARNING)

def optimizer_optuna(n_trials,number_of_random_points,optimizer_direction,optuna_objective):
 
    use_sampler = TPESampler(n_startup_trials=number_of_random_points)

    study = optuna.create_study(sampler=use_sampler, direction=optimizer_direction)
    study.optimize(optuna_objective,n_trials=n_trials, show_progress_bar = True,gc_after_trial=True)
 
    print('best_params:',study.best_trial.params,
              'best_score:',study.best_trial.values,
              '\n')
 
    return study.best_trial.params, study.best_trial.values
 
n_trials = 50

import warnings
warnings.filterwarnings('ignore',message='The objective has been evaluated at this point before trails')
optuna.logging.set_verbosity(optuna.logging.ERROR)
best_params, best_score = optimizer_optuna(n_trials,number_of_random_points,optimizer_direction,optuna_objective)
 

In [ ]:
best_model = CatBoostClassifier(**best_params,verbose=False,task_type='GPU') 
best_model.fit(X_train, y_train) 

pickle.dump(best_model, open("./results/root_catboost_best_model.pkl", "wb"))

#Print scores for Multiclass 
y_pred_1 = best_model.predict(X_test) 
y_prob_1 = best_model.predict_proba(X_test) 
f1 = f1_score(y_test, y_pred_1, average='macro')
acc = accuracy_score(y_test, y_pred_1)
print('f1 : ', f1)
print('accuracy : ', acc)

print(classification_report(y_test, y_pred_1, digits=3)) 
print('Accuracy score: ', accuracy_score(y_test, y_pred_1)) 
print('Roc auc score : ', roc_auc_score(y_test, y_prob_1, multi_class='ovr'))

skplt.metrics.plot_roc(y_test, y_prob_1)
plt.show()

In [ ]:
best_model = pickle.load(open("./results/root_catboost_best_model.pkl", "rb"))

In [ ]:
# Note: the reference genome most be MSU7 from Ensembl Plants (https://plants.ensembl.org)
def preprocessing_test(adataPath, hvgs_names):
    adata = sc.read_10x_mtx(adataPath,var_names= 'gene_symbols',cache = False)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    intersection_genes = list(set(adata.var_names) & set(hvgs_names))
    adata = adata[:, intersection_genes]
    # Scale
    sc.pp.scale(adata, max_value=10)
    X_df = pd.DataFrame(adata.X, index=adata.obs_names, columns=adata.var_names)
    complement_gene = list(set(hvgs_names) - set(X_df.columns))  
    final_df = X_df.assign(**{col: 0 for col in complement_gene})
    print("The count of model features is: ", len(hvgs_names))
    print("The count of genes that intersect with the model features in the new dataset is: ", len(list(set(hvgs_names) & set(X_df.columns))))
    print(type(final_df), final_df.shape)
    return final_df

test_root_df = preprocessing_test(adataPath = "./test_data/filtered_feature_bc_matrix",
                             hvgs_names = highly_variable_genes)

In [ ]:
predicted_labels_catboost = best_model.predict(test_root_df) 
y_probas_catboost = best_model.predict_proba(test_root_df)
result_df_catboost = pd.DataFrame({'Predicted_Label': predicted_labels_catboost.flatten()}, index=test_root_df.index)
print(collections.Counter(result_df_catboost['Predicted_Label']))

In [ ]:
result_df_catboost.to_csv('./results/Predicted_Label.csv', index=False)